In [10]:
import pandas as pd
import os
import re
from tqdm.auto import tqdm
import numpy as np
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
import re

pd.set_option('display.max_columns', None)

In [11]:
folder = '/home/haoran/Desktop/BERT_DeBias Aux Code/Camera Ready Results/vr_downstream'
numFolds= 1000

In [12]:
sheets = ['overall', 'gender' ,'language', 'insurance', 'ethnicity'] 
allowed_models = {'overall': ['baseline', 'debiased_gender', 'debiased_insurance', 'debiased_ethnicity', 'debiased_language'],
                 'gender': ['baseline', 'debiased_gender'],
                 'language': ['baseline', 'debiased_language'],
                 'insurance': ['baseline', 'debiased_insurance'],
                 'ethnicity': ['baseline', 'debiased_ethnicity']}

In [13]:
def gap_significant(lower, upper):
    return (((lower <0) & (upper < 0)) | ((lower >0) & (upper > 0)))

In [14]:
def add_to_dict(gap_dict, model, sheet, name, num_sig, num_favor):
    if model not in gap_dict:
        gap_dict[model] = {}
    if sheet not in gap_dict[model]:
        gap_dict[model][sheet] = {}
    gap_dict[model][sheet][name] = [num_sig, num_favor]

def add_gap(model, gap, sheet, res):
    res[model+'_' + gap+'_sig'] = multipletests(res[model+'_'+gap + '_p'], alpha = 0.05, method = "fdr_bh")[0]  
    add_to_dict(gap_infos_hyp, model, sheet, gap, res[model+'_' + gap+'_sig'].sum(), 
                       np.isclose(res.loc[res[model+'_' + gap+'_sig'], model+'_'+gap+'_favor'],  1).sum())
    add_to_dict(gap_infos_naive, model, sheet, gap, res[model+'_' + gap + 'naive_sig'].astype(int).sum(),
                       (res.loc[res[model+'_' + gap + 'naive_sig'], model+'_'+gap] > 0).astype(int).sum() )

In [15]:
dfs = {}
gap_infos_hyp = {}
gap_infos_naive = {}
for sheet in sheets:
    res = pd.DataFrame()
    for root, dirs, files in tqdm(os.walk(folder)):
        for d in dirs:
            mname = d
            for file in os.scandir(os.path.join(root, d)):
                if not(file.name.endswith('results.xlsx')) or (file.name.startswith('~')):
                    continue
                    
                if 'ADV_PT' in mname or '_adv_' in mname:
                    if 'gender' in mname:
                        model = 'debiased_gender'
                    elif 'insurance' in mname:
                        model = 'debiased_insurance'
                    elif 'language' in mname:
                        model = 'debiased_language'
                    elif 'ethnicity' in mname:
                        model = 'debiased_ethnicity'
                else:
                    model = 'baseline'
                if model not in allowed_models[sheet]:
                    break

                if '312mortality' in mname:
                    target = '312mortality'
                    continue
                elif 'inhosp_mort' in mname:
                    target = 'inhosp_mort'
                elif 'phenotype' in mname:
                    name = re.findall(r'.*512_(?:lambda1_)*(.*)', mname)[0]
                    if name.endswith('_gs'):
                        name = name[:-3]
                    name = name.replace('_', ' ')
                    if 'phenotype_all' in mname:
                        target = 'phenotype_all_%s' % name
                    else:
                        target = 'phenotype_first_%s' % name
                else:
                    continue

                if sheet == 'overall':
                    df = pd.read_excel(os.path.join(root, file), index_col=0, sheet_name = 'all')
                    columns = ['all_auroc', 'all_auprc', 'all_recall', 'all_class_true_count']
                    for i in columns:
                        res.loc[target, model + '_' +i] = df.loc[i, 'avg']
                        res.loc[target, model + '_' +i+'lowerCI'] = df.loc[i, '2.5%']
                        res.loc[target, model + '_' +i+'upperCI'] = df.loc[i, '97.5%']

                elif sheet == 'gender':
                    df = pd.read_excel(os.path.join(root, file), index_col=0, sheet_name = 'gender')
                    columns = {
                            'gender=="M"_dgap_max':'Parity Gap (M-F)',
                               'gender=="M"_egap_positive_max':'Recall Gap',
                               'gender=="M"_egap_negative_max':'Specificity Gap'
                              }
                    for i in columns:
                        res.loc[target, model + '_' +columns[i]] = df.loc[i, 'avg']
                        res.loc[target, model + '_' +columns[i] + '_p'] = df.loc[i, 'p']
                        res.loc[target, model + '_' +columns[i] + '_favor'] = df.loc[i, 'favor']
                        
                        res.loc[target, model + '_' +columns[i]+'lowerCI'] = df.loc[i, '2.5%']
                        res.loc[target, model + '_' +columns[i]+'upperCI'] = df.loc[i, '97.5%']
                        res.loc[target, model + '_' +columns[i]+'naive_sig'] = gap_significant(df.loc[i, '2.5%'], df.loc[i, '97.5%'])
                
                elif sheet == 'language':
                    df = pd.read_excel(os.path.join(root, file), index_col=0, sheet_name = 'language_to_use')
                    columns = {'language_to_use=="English"_dgap_max' : 'Parity Gap (E-O)', 
                               'language_to_use=="English"_egap_positive_max' : 'Recall Gap', 
                               'language_to_use=="English"_egap_negative_max' : 'Specificity Gap' }
                    for i in columns:
                        res.loc[target, model + '_' +columns[i]] = df.loc[i, 'avg']
                        res.loc[target, model + '_' +columns[i] + '_p'] = df.loc[i, 'p']
                        res.loc[target, model + '_' +columns[i] + '_favor'] = df.loc[i, 'favor']
                        
                        res.loc[target, model + '_' +columns[i]+'lowerCI'] = df.loc[i, '2.5%']
                        res.loc[target, model + '_' +columns[i]+'upperCI'] = df.loc[i, '97.5%']
                        res.loc[target, model + '_' +columns[i]+'naive_sig'] = gap_significant(df.loc[i, '2.5%'], df.loc[i, '97.5%'])
                                              
                elif sheet == 'insurance':                            
                    df = pd.read_excel(os.path.join(root, file), index_col=0, sheet_name = 'insurance')
                    columns = []
                    for i in ['Medicare', 'Private', 'Medicaid']:
                        for j in ['dgap_max', 'egap_positive_max', 'egap_negative_max']:
                            columns.append(
                                'insurance=="%s"_%s'%(i,j)
                            )
                    for i in columns:
                        n = model+ '_' + i.replace('insurance==', '')
                        res.loc[target, n] = df.loc[i, 'avg']
                        res.loc[target, n + '_p'] = df.loc[i, 'p']
                        res.loc[target, n + '_favor'] = df.loc[i, 'favor']
                        
                        res.loc[target, n+'lowerCI'] = df.loc[i, '2.5%']
                        res.loc[target, n+'upperCI'] = df.loc[i, '97.5%']
                        if 'dgap_' in n or 'egap_' in n:
                            res.loc[target, n+'naive_sig'] = gap_significant(df.loc[i, '2.5%'], df.loc[i, '97.5%'])


                elif sheet == 'ethnicity':
                    df = pd.read_excel(os.path.join(root, file), index_col=0, sheet_name = 'ethnicity_to_use')
                    columns = []
                    for i in ['WHITE', 'BLACK', 'ASIAN', 'HISPANIC/LATINO', 'OTHER']:
                        for j in ['dgap_max', 'egap_positive_max', 'egap_negative_max']:
                            columns.append(
                                'ethnicity_to_use=="%s"_%s'%(i,j)
                            )
                    for i in columns:
                        n = model+ '_' + i.replace('ethnicity_to_use==', '')
                        res.loc[target, n] = df.loc[i, 'avg']
                        res.loc[target, n + '_p'] = df.loc[i, 'p']
                        res.loc[target, n + '_favor'] = df.loc[i, 'favor']
                
                        res.loc[target, n+'lowerCI'] = df.loc[i, '2.5%']
                        res.loc[target, n+'upperCI'] = df.loc[i, '97.5%']
                        if 'dgap_' in n or 'egap_' in n:
                            res.loc[target, n+'naive_sig'] = gap_significant(df.loc[i, '2.5%'], df.loc[i, '97.5%'])
    
    
    if sheet == 'gender':
        for m in allowed_models[sheet]:
            for i in ('Parity Gap (M-F)','Recall Gap','Specificity Gap'):
                add_gap(m, i, sheet, res)
                
    if sheet == 'language':
        for m in allowed_models[sheet]:
            for i in ('Parity Gap (E-O)','Recall Gap','Specificity Gap'):
                add_gap(m, i, sheet, res)
    
    if sheet == 'insurance':
        for m in allowed_models[sheet]:
            for g in ['Medicare', 'Private', 'Medicaid']:
                for i in ('"%s"_'%g + t for t in ['dgap_max','egap_positive_max','egap_negative_max']):
                    add_gap(m, i, sheet, res)
                    
    if sheet == 'ethnicity':
        for m in allowed_models[sheet]:
            for g in ['WHITE', 'BLACK', 'ASIAN', 'HISPANIC/LATINO', 'OTHER']:
                for i in ('"%s"_'%g + t for t in ['dgap_max','egap_positive_max','egap_negative_max']):
                    add_gap(m, i, sheet, res)
    
    dfs[sheet] = res.sort_index()

In [7]:
dict_of_df = {k: pd.DataFrame(v) for k,v in gap_infos_naive.items()}
naive_df = pd.concat(dict_of_df, axis=1, sort = False)

In [8]:
dict_of_df = {k: pd.DataFrame(v) for k,v in gap_infos_hyp.items()}
hyp_df = pd.concat(dict_of_df, axis=1, sort = False)

### Naive

In [9]:
def display_tables(df):
    for i in ['gender', 'language', 'ethnicity', 'insurance']:
        temp = df.T.xs(i, level = 1).dropna(axis = 1)
        temp = temp.apply(lambda x: x.apply(lambda y: str(y[0]) + ' (' + "{:.0%}".format(y[1]/y[0]) + ')'), axis = 0)
        if i in ['ethnicity', 'insurance']:
            temp = temp.T    
            temp['Gap' ] = list(map(lambda x: list(reversed(re.split(r'"_', x)))[0][:-4], temp.index))
            temp['Group' ] = list(map(lambda x: list(reversed(re.split(r'"_', x)))[1][1:].lower(), temp.index))
            temp = temp.set_index(['Gap','Group']).sort_index()        
        display(temp)
        if i in ['ethnicity', 'insurance']:
            temp = temp[['baseline']].reset_index()
            temp = temp.pivot_table(values = 'baseline', index = 'Group', columns = 'Gap', aggfunc = lambda x: x)
            temp = temp[['egap_positive', 'dgap', 'egap_negative']]
            if i =='ethnicity':
                temp = temp.loc[['white','black','hispanic/latino','asian', 'other']]
            elif i =='insurance':
                temp = temp.loc[['medicare', 'private', 'medicaid']]
            display(temp)

In [10]:
display_tables(naive_df)

,Parity Gap (M-F),Recall Gap,Specificity Gap
baseline,25 (36%),13 (62%),20 (80%)
debiased_gender,25 (36%),9 (56%),20 (70%)


,Parity Gap (E-O),Recall Gap,Specificity Gap
baseline,17 (12%),7 (29%),9 (89%)
debiased_language,20 (20%),7 (43%),11 (73%)


baseline debiased_ethnicity
Gap           Group                                        
dgap          asian             7 (100%)           4 (100%)
              black             18 (72%)           24 (71%)
              hispanic/latino    18 (0%)            17 (6%)
              other               8 (0%)             6 (0%)
              white             22 (82%)           20 (85%)
egap_negative asian              8 (50%)            7 (43%)
              black             11 (18%)            11 (9%)
              hispanic/latino  20 (100%)          13 (100%)
              other             9 (100%)           7 (100%)
              white             12 (17%)           12 (17%)
egap_positive asian              8 (62%)            7 (29%)
              black              5 (20%)            7 (71%)
              hispanic/latino     7 (0%)             6 (0%)
              other              10 (0%)            11 (9%)
              white              4 (75%)           6 (100%)

Gap,egap_positive,dgap,egap_negative
Group,,,
white,4 (75%),22 (82%),12 (17%)
black,5 (20%),18 (72%),11 (18%)
hispanic/latino,7 (0%),18 (0%),20 (100%)
asian,8 (62%),7 (100%),8 (50%)
other,10 (0%),8 (0%),9 (100%)


baseline debiased_insurance
Gap           Group                                
dgap          medicaid  31 (19%)           32 (12%)
              medicare  51 (92%)           49 (94%)
              private    41 (2%)            41 (2%)
egap_negative medicaid  30 (83%)           29 (83%)
              medicare   48 (6%)            46 (2%)
              private   40 (98%)           41 (95%)
egap_positive medicaid  20 (20%)            16 (0%)
              medicare  33 (85%)           34 (88%)
              private    15 (7%)           14 (14%)

Gap,egap_positive,dgap,egap_negative
Group,,,
medicare,33 (85%),51 (92%),48 (6%)
private,15 (7%),41 (2%),40 (98%)
medicaid,20 (20%),31 (19%),30 (83%)


### Multiple Hypothesis

In [11]:
display_tables(hyp_df)

,Parity Gap (M-F),Recall Gap,Specificity Gap
baseline,23 (30%),6 (67%),16 (81%)
debiased_gender,24 (38%),4 (75%),17 (71%)


,Parity Gap (E-O),Recall Gap,Specificity Gap
baseline,14 (0%),1 (100%),3 (100%)
debiased_language,13 (23%),5 (60%),0 (nan%)


baseline debiased_ethnicity
Gap           Group                                        
dgap          asian             1 (100%)           1 (100%)
              black             14 (71%)           24 (71%)
              hispanic/latino    15 (0%)            11 (9%)
              other               2 (0%)             5 (0%)
              white             17 (76%)           16 (81%)
egap_negative asian              6 (50%)            6 (50%)
              black              9 (22%)            6 (17%)
              hispanic/latino  16 (100%)           7 (100%)
              other             5 (100%)           7 (100%)
              white              9 (22%)            5 (20%)
egap_positive asian              6 (50%)            7 (29%)
              black               2 (0%)            4 (50%)
              hispanic/latino     7 (0%)             6 (0%)
              other              10 (0%)            10 (0%)
              white             0 (nan%)           1 (100%)

Gap,egap_positive,dgap,egap_negative
Group,,,
white,0 (nan%),17 (76%),9 (22%)
black,2 (0%),14 (71%),9 (22%)
hispanic/latino,7 (0%),15 (0%),16 (100%)
asian,6 (50%),1 (100%),6 (50%)
other,10 (0%),2 (0%),5 (100%)


baseline debiased_insurance
Gap           Group                                
dgap          medicaid  32 (19%)           34 (12%)
              medicare  51 (92%)           52 (94%)
              private    41 (2%)            44 (5%)
egap_negative medicaid  30 (83%)           29 (83%)
              medicare   48 (6%)            48 (4%)
              private   44 (98%)           42 (95%)
egap_positive medicaid  19 (16%)            12 (0%)
              medicare  35 (83%)           35 (86%)
              private    12 (8%)            12 (8%)

Gap,egap_positive,dgap,egap_negative
Group,,,
medicare,35 (83%),51 (92%),48 (6%)
private,12 (8%),41 (2%),44 (98%)
medicaid,19 (16%),32 (19%),30 (83%)


## Write to file

In [16]:
with pd.ExcelWriter('results.xlsx') as writer:
    for sheet, df in dfs.items():
        df.to_excel(writer, sheet_name = sheet)

In [236]:
# gender
cols = {'Recall Gap': 'Positive Equality Gap', 
        'Specificity Gap': 'Negative Equality Gap', 
        'Parity Gap (M-F)': 'Parity Gap'}
sig = dfs['gender'][dfs['gender']['baseline_Recall Gapnaive_sig']]
sig = sig.loc[sig['baseline_Recall Gap'].sort_values(ascending = False).index]
final = None
for i,b in cols.items():    
    temp = sig[['baseline_%s'%i, 'baseline_%slowerCI'%i, 'baseline_%supperCI'%i]]
    temp.columns = ['Gap', 'Lower', 'Upper']
    temp['Type'] =b 
    if final is None:
        final = temp
    else:
        final = final.append(temp)
with pd.ExcelWriter('gender_toplot.xlsx') as writer:
    final.to_excel(writer)    

/home/haoran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [237]:
# language
cols = {'Recall Gap': 'Positive Equality Gap', 
        'Specificity Gap': 'Negative Equality Gap', 
        'Parity Gap (E-O)': 'Parity Gap'}
sig = dfs['language'][dfs['language']['baseline_Recall Gapnaive_sig']]
sig = sig.loc[sig['baseline_Recall Gap'].sort_values(ascending = False).index]
final = None
for i,b in cols.items():    
    temp = sig[['baseline_%s'%i, 'baseline_%slowerCI'%i, 'baseline_%supperCI'%i]]
    temp.columns = ['Gap', 'Lower', 'Upper']
    temp['Type'] =b 
    if final is None:
        final = temp
    else:
        final = final.append(temp)
with pd.ExcelWriter('language_toplot.xlsx') as writer:
    final.to_excel(writer)    

/home/haoran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


## AUROC Table

In [78]:
mapping = pd.read_csv('./bias graphs//Shortform_mapping.csv').set_index('name').to_dict()['short']
for a,b in mapping.copy().items():
    if a.startswith('phenotype_all'):
        mapping['phenotype_all_'+ a[len('phenotype_all '):].replace('_', ' ')] = b
    elif a.startswith('phenotype_first'):
        mapping['phenotype_first_'+ a[len('phenotype_first '):].replace('_', ' ')] = b

In [79]:
aurocs = dfs['overall'][['baseline_all_auroc','baseline_all_auprc','debiased_gender_all_auroc','debiased_gender_all_auprc']]

In [80]:
aurocs.columns = ['Baseline AUROC','Baseline AUPRC','Debiased AUROC','Debiased AUPRC']

In [81]:
aurocs = aurocs.reset_index().rename(columns = {'index':'Task'})

In [82]:
# aurocs['Task'] = pd.Series(aurocs['Task']).map(mapping)   

In [88]:
def sep_ind(x):
    if x.startswith('phenotype_all'):
        return ['Phenotype All', x[len('phenotype_all_'):]]
    elif x.startswith('phenotype_first'):
        return ['Phenotype First', x[len('phenotype_first_'):]]
    else:
        return ['', 'In-hospital Mortality']
temp = aurocs['Task'].apply(lambda x: sep_ind(x))
aurocs['Type'], aurocs['Task'] = [i[0] for i in temp], [i[1] for i in temp]

In [92]:
aurocs = aurocs.set_index(['Type', 'Task'])

In [96]:
print(aurocs.to_latex(formatters={i: lambda x: '{:.2%}'.format(x) for i in aurocs.columns}))

\begin{tabular}{llrrrr}
\toprule
                &             & Baseline AUROC & Baseline AUPRC & Debiased AUROC & Debiased AUPRC \\
Type & Task &                &                &                &                \\
\midrule
                & In-hospital Mortality &         86.74\% &         50.17\% &         85.96\% &         48.55\% \\
Phenotype All & Acute and unspecified renal failure &         83.64\% &         57.89\% &         83.37\% &         57.87\% \\
                & Acute cerebrovascular disease &         93.51\% &         61.97\% &         92.60\% &         61.87\% \\
                & Acute myocardial infarction &         88.16\% &         63.29\% &         88.27\% &         63.42\% \\
                & Cardiac dysrhythmias &         77.87\% &         64.05\% &         77.02\% &         63.28\% \\
                & Chronic kidney disease &         86.06\% &         53.83\% &         86.08\% &         54.17\% \\
                & Chronic obstructive pulmonary disease an